In [1]:
using Revise
using VariPEPS
using TensorKit
using OptimKit
using JLD2 #needed?
using LinearAlgebra
using Random

In this notebook we show the very basic functionality of the workhorse funtions of the `VariPEPS.jl` code

Specify the Bond-dimension `d`, the environment-bond-dimension `χ`. 

In [2]:
χ = 8
d = 2

2

Here specify the unit cell. For this we define an array "Pattern_arr" that represents the unit cell structure.\
The structure is [x-coodinate, y-coordinate].\
If two positions within the Pattern_arr are filled with the same number, we assume the tensors at these positions to be identical.\
The choice below corresponds to a unit cell like:   
     --> x \
    |     A   B \
   v      B   A \
   y
   

Note the convention used here 

In [3]:
Pattern_arr = Array{Any}(undef,2,2)

Pattern_arr[1,1] = 1
Pattern_arr[2,1] = 2
Pattern_arr[1,2] = 2
Pattern_arr[2,2] = 1;

For our first example we are going to look at the Heisenberg-antiferromagnet on the square lattice.\
Here we specify the keywords corresponding to this choice of model and lattice on in a tuple.

Regarding the keywords:\
**lattice**: tells us which lattice we use.\
**model**: which Hamiltonian are we using.\
**Ham_parameters**: Give values for the parameters in the Hamiltonian. Here we choose an anti-ferromagnetic model (J=1) with no magnetic field term (h = 0) which points in no direction (dir = [0,0,0])\
**Projector_type**: During the CTMRG algorithm we need projectors. These can be choosen in different ways. Here we choose the :half option. An other more expensive choice would be :fullfishman.\
**Space_type**: This tells us what the basic vector space is that we want to work with. As we have a TRS model here we can choose a real wavefunction so we choose ℝ. Otherwise choose ℂ.\

In [4]:
keywords = (lattice = :square, model = :Heisenberg_square, Ham_parameters = (J = 1, h = 0 , dir = [0,0,0]), Projector_type = :half, Space_type = :Z2, svd_type = :full_trunc)

(lattice = :square, model = :Heisenberg_square, Ham_parameters = (J = 1, h = 0, dir = [0, 0, 0]), Projector_type = :half, Space_type = :Z2, svd_type = :full_trunc)

In [5]:
#here I define some Z2 tensors

V_phys = ℤ₂Space(0=>1, 1=>1)
V_virt = ℤ₂Space(0=>2, 1=>2)

seed = 1235
rng = MersenneTwister(seed)
randn_with_seed = (tuple) -> rand(rng, Float64, tuple)


A = TensorMap(randn_with_seed, V_virt ⊗ V_virt ← V_virt ⊗ V_virt ⊗ (V_phys)')
B = TensorMap(randn_with_seed, V_virt ⊗ V_virt ← V_virt ⊗ V_virt ⊗ (V_phys)')
C = TensorMap(randn_with_seed, V_virt ⊗ V_virt ← V_virt ⊗ V_virt ⊗ (V_phys)')
D = TensorMap(randn_with_seed, V_virt ⊗ V_virt ← V_virt ⊗ V_virt ⊗ (V_phys)')

shift = TensorMap(ones, Float64, V_virt ⊗ V_virt ← V_virt ⊗ V_virt ⊗ (V_phys)')
A = A #+ 2*shift
B = B #+ 6*shift
C = C #+ 6*shift
D = D #+ 6*shift

A = A / norm(A)
B = B / norm(B)
C = C / norm(C)
D = D / norm(D)

#A_rot1 = permute(A, (2,3)(4,1,5))

loc_in_Z2 = [A,B]

2-element Vector{TensorMap{GradedSpace{Z2Irrep, Tuple{Int64, Int64}}, 2, 3, Z2Irrep, TensorKit.SortedVectorDict{Z2Irrep, Matrix{Float64}}, FusionTree{Z2Irrep, 2, 0, 1, Nothing}, FusionTree{Z2Irrep, 3, 1, 2, Nothing}}}:
 TensorMap((Rep[ℤ₂](0=>2, 1=>2) ⊗ Rep[ℤ₂](0=>2, 1=>2)) ← (Rep[ℤ₂](0=>2, 1=>2) ⊗ Rep[ℤ₂](0=>2, 1=>2) ⊗ Rep[ℤ₂](0=>1, 1=>1)')):
* Data for sector (Irrep[ℤ₂](0), Irrep[ℤ₂](0)) ← (Irrep[ℤ₂](1), Irrep[ℤ₂](1), Irrep[ℤ₂](0)):
[:, :, 1, 1, 1] =
 0.06958157389810678  0.0034472816938683516
 0.09630343414767317  0.06270778455826177

[:, :, 2, 1, 1] =
 0.06490972741526485   0.10121310705357806
 0.007478155778288439  0.07391161101088865

[:, :, 1, 2, 1] =
 0.03255439243288503  0.04059587808788047
 0.09314381622221315  0.05306656128663306

[:, :, 2, 2, 1] =
 0.038986711648773355  0.0009710777010966978
 0.048701945199097665  0.06932162258037605
* Data for sector (Irrep[ℤ₂](1), Irrep[ℤ₂](1)) ← (Irrep[ℤ₂](1), Irrep[ℤ₂](1), Irrep[ℤ₂](0)):
[:, :, 1, 1, 1] =
 0.008613995901100487  0.0984694

In [ ]:
ctmrg_res = ctmrg(loc_in_Z2, 16, Pattern_arr; keywords..., observ = false, conv_info = true, output_envs = true, maxiter = 300)#, reuse_envs = old_envs) #adjust_χ = (10^-3,15)

In [ ]:
e_new, gr_new = energy_and_gradient(loc_in_Z2, 16, Pattern_arr; keywords..., conv_info = true, maxiter_pre = 250)

Now we should the above functions to variationally find the best d=2 approximation to the ground state of the Heisenberg antiferromagnet.

For this we can use gradient based optimizers like L-BFGS. They are implemented in different packages, here we use OptimKit.jl. (an alternative would be Optim.jl)\
for the optimization we need the following two functions:

In [ ]:
function _finalize!(x,f,g, numiter)
    
    
    flush(stdout) #this just makes it so the output does not appear all at once.
    flush(stderr)

    
    #here we could save e.g. intermediate output.
    
    return x, f, g
end

function _inner(x, ξ_1, ξ_2)
    return dot(real(ξ_1), real(ξ_2)) + dot(imag(ξ_1), imag(ξ_2))
end

we specify the parameters that are used in the optimization. 

In [ ]:
optimparas1 = (gradtol = 1e-5, verbosity = 2, maxiter = 50, linesearch = HagerZhangLineSearch(; c₁ = 1e-4, c₂ = 0.9, maxiter = 40))
kwargs = (finalize! = _finalize!, inner = _inner)

Here we call the function that optimizes based on our gradient function.\
This function returns the approximation to the ground state `x` found in this procedure, the ground state energy `e` as well as information on the convergence.\
Below we plot some of the results.

In [ ]:
x1, e, gr, numfg, info = optimize(x -> energy_and_gradient(x, 16, Pattern_arr, keywords; adjust_χ = false), loc_in_Z2, LBFGS(15; optimparas1... ); kwargs... )

UndefVarError: UndefVarError: `x1` not defined

In [18]:
using Plots

In [19]:
pval = plot(5:1:51, info[5:end,1], color = :blue, legend = false, ylabel = "⟨E⟩", xlabel = "iterations");

pgrad = plot(5:1:51, log10.(info[5:end,2]), color = :red, ylabel = " log10(‖∇E‖)", legend = false);


UndefVarError: UndefVarError: `info` not defined

In [20]:
pval

UndefVarError: UndefVarError: `pval` not defined

In [21]:
pgrad

UndefVarError: UndefVarError: `pgrad` not defined